In [2]:
import pandas as pd
import numpy as np

In [3]:
file = '2024-03-04_SUEK_SAP_Debt.xlsx'

In [4]:
!dir

 ��� � ���ன�⢥ C ����� ���� ULayeredImage
 ��਩�� ����� ⮬�: 80F9-578F

 ����ন��� ����� c:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Debt

04.03.2024  09:16    <DIR>          .
04.03.2024  09:16    <DIR>          ..
04.03.2024  09:14            36�010 2024-03-04_SUEK_SAP_Debt.xlsx
29.02.2024  16:51            37�746 Copies.ipynb
19.02.2024  15:42           939�052 Credit_portfolio_SAP.xlsx
04.03.2024  09:14            12�817 Debt_SUEK.ipynb
               4 䠩���      1�025�625 ����
               2 �����  26�904�481�280 ���� ᢮�����


In [5]:
data_export = pd.read_excel(file)
data_export["index_copy"] = data_export.index
data_export = data_export.rename(columns={'Entity group':'Entity_group', 'Counterparty Group':'Counterparty_Group', 'amoutn outstanding':'amoutn_outstanding', 'amount USD eq': 'amount_USD_eq'})

In [6]:
data_export = data_export.query('Entity_group != "External" & Counterparty_Group != "External"')
# филтрация данных на внутренние операции
intersection_list = list(set(data_export.Entity.unique()).intersection(set(data_export.Counterparty.unique())))
# список значений, где Entity и Counterparty пересекаются

In [7]:
not_intersection_data = data_export[~(data_export['Entity'].isin(intersection_list) & data_export['Counterparty'].isin(intersection_list))]
# выбираю только те данные, где Entity и Counterparty не пересекаются (либо Entity не должно быть в списке, либо Counterparty)

not_intersection_data['abs_amoutn_outstanding'] = not_intersection_data['amoutn_outstanding'].abs()

data_index_list = not_intersection_data.groupby(['abs_amoutn_outstanding','termEnd'], as_index=False)['index_copy']\
.apply(list)\
.rename(columns={'index_copy':'data_export_index_list'})
data_index_list['index_count'] = data_index_list['data_export_index_list'].apply(len)
data_douple = data_index_list.query('index_count > 1')

douple_srings_list = list(np.concatenate(data_douple.data_export_index_list.values, axis = 0))
douple_srings_data = data_export.loc[douple_srings_list]
douple_srings_data.shape

douple_list = douple_srings_list

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_10360\3203060371.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_intersection_data['abs_amoutn_outstanding'] = not_intersection_data['amoutn_outstanding'].abs()


In [8]:
intersection_data = data_export[(data_export['Entity'].isin(intersection_list) & data_export['Counterparty'].isin(intersection_list))]
# выбираю только те данные, где Entity и Counterparty не пересекаются (либо Entity не должно быть в списке, либо Counterparty)

intersection_data['abs_amoutn_outstanding'] = intersection_data['amoutn_outstanding'].abs()

data_index_list_2 = intersection_data.groupby(['abs_amoutn_outstanding','termEnd'], as_index=False)['index_copy']\
.apply(list)\
.rename(columns={'index_copy':'data_export_index_list'})
data_index_list_2['index_count'] = data_index_list_2['data_export_index_list'].apply(len)
data_douple_2 = data_index_list_2.query('index_count > 1')

douple_srings_list_2 = list(np.concatenate(data_douple_2.data_export_index_list.values, axis = 0))
douple_srings_data_2 = data_export.loc[douple_srings_list_2]

douple_list = douple_list + douple_srings_list_2

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_10360\3875329925.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intersection_data['abs_amoutn_outstanding'] = intersection_data['amoutn_outstanding'].abs()


In [9]:
data_not_copy = data_export[~(data_export['index_copy'].isin(douple_list))]

In [10]:
with pd.ExcelWriter(file, engine='openpyxl', mode='a') as writer:
    douple_srings_data.to_excel(writer,sheet_name='Copies', index=False)
    douple_srings_data_2.to_excel(writer,sheet_name='Copies_same', index=False)
    data_not_copy.to_excel(writer,sheet_name='Not_copies', index=False)

In [11]:
more_then_2_strings_list = data_index_list.query('index_count > 2').data_export_index_list.values[0]
two_strings_data = douple_srings_data[~(douple_srings_data['index_copy'].isin(more_then_2_strings_list))].reset_index(drop = True)
names_dict = {}
for i in range(0,len(two_strings_data)-1):
    if i%2 == 0:
        names_dict[two_strings_data.iloc[i, 0]] = []
        names_dict[two_strings_data.iloc[i, 0]].append(two_strings_data.iloc[i+1, 2])
        names_dict[two_strings_data.iloc[i, 0]] = [x for x in set(names_dict[two_strings_data.iloc[i, 0]])]
    else:
        names_dict[two_strings_data.iloc[i, 0]] = []
        names_dict[two_strings_data.iloc[i, 0]].append(two_strings_data.iloc[i-1, 2])
        names_dict[two_strings_data.iloc[i, 0]] = [x for x in set(names_dict[two_strings_data.iloc[i, 0]])]


In [12]:
more_then_2_strings_data = douple_srings_data[douple_srings_data['index_copy'].isin(more_then_2_strings_list)].reset_index(drop = True)
more_then_2_strings_data

,Entity,Entity_group,Counterparty,Counterparty_Group,amoutn_outstanding,Currency,amount_USD_eq,termEnd,Days,Period,dealClass1,dealClass2,instrumentOwner,dealSet,facility,Source,index_copy
0,"ООО ""Сибирская генерирующая компания""",SGC,"АО ""СИБЭКО""",SGC,-1.000000e+10,RUB,-103090870.0,2028-06-14,1627,2Y+,NaN,NaN,NaN,NaN,NaN,SAP,68
1,"ООО ""Сибирская генерирующая компания""",SGC,"АО""БарнаульскаяГенерация""",SGC,-1.000000e+10,RUB,-103090870.0,2028-06-14,1627,2Y+,NaN,NaN,NaN,NaN,NaN,SAP,171
2,"АО ""Барнаульская генерация""",SGC,"ООО ""СГК""",SGC,1.000000e+10,RUB,103090870.0,2028-06-14,1627,2Y+,NaN,NaN,NaN,NaN,NaN,SAP,264
3,"Акционерное общество ""Сибирская энергетическая...",SGC,"ООО ""СГК""",SGC,1.000000e+10,RUB,103090870.0,2028-06-14,1627,2Y+,NaN,NaN,NaN,NaN,NaN,SAP,282


Проверка вхождения лишь одного значения напротив ключа:

In [13]:
ltn_list = []
for i in names_dict.values():
    ltn_list.append(len(i))
if (pd.Series(ltn_list) > 1).any() or (pd.Series(ltn_list) == 0).any():
    raise KeyError

In [14]:
names_dictionary = dict(zip(names_dict.keys(), [i[0] for i in names_dict.values()]))
with pd.ExcelWriter(file, engine='openpyxl', mode='a') as writer:
    # douple_srings_data.to_excel(writer,sheet_name='Copies', index=False)
    # douple_srings_data_2.to_excel(writer,sheet_name='Copies_same', index=False)
    # data_not_copy.to_excel(writer,sheet_name='Not_copies', index=False)
    pd.DataFrame({'keys': names_dictionary.keys(), 'values': names_dictionary.values()}).to_excel(writer,sheet_name='Copies_dict', index=False)

Удаление дубликатов:

In [15]:
names_dictionary
douple_srings_data['Entity'] = douple_srings_data['Entity'].replace(names_dictionary)
douple_srings_data[douple_srings_data['index_copy'].isin(more_then_2_strings_list)]


,Entity,Entity_group,Counterparty,Counterparty_Group,amoutn_outstanding,Currency,amount_USD_eq,termEnd,Days,Period,dealClass1,dealClass2,instrumentOwner,dealSet,facility,Source,index_copy
68,"ООО ""СГК""",SGC,"АО ""СИБЭКО""",SGC,-1.000000e+10,RUB,-103090870.0,2028-06-14,1627,2Y+,NaN,NaN,NaN,NaN,NaN,SAP,68
171,"ООО ""СГК""",SGC,"АО""БарнаульскаяГенерация""",SGC,-1.000000e+10,RUB,-103090870.0,2028-06-14,1627,2Y+,NaN,NaN,NaN,NaN,NaN,SAP,171
264,"АО""БарнаульскаяГенерация""",SGC,"ООО ""СГК""",SGC,1.000000e+10,RUB,103090870.0,2028-06-14,1627,2Y+,NaN,NaN,NaN,NaN,NaN,SAP,264
282,"АО ""СИБЭКО""",SGC,"ООО ""СГК""",SGC,1.000000e+10,RUB,103090870.0,2028-06-14,1627,2Y+,NaN,NaN,NaN,NaN,NaN,SAP,282


In [16]:
douple_srings_data.head(2)

,Entity,Entity_group,Counterparty,Counterparty_Group,amoutn_outstanding,Currency,amount_USD_eq,termEnd,Days,Period,dealClass1,dealClass2,instrumentOwner,dealSet,facility,Source,index_copy
22,МК «СУЭК ЛТД» (ООО),SPV,"АО ""Дальтрансуголь""",NTC,-2000000.0,USD,-2000000.0,2024-08-30,243,6M-1Y,NaN,NaN,NaN,NaN,NaN,SAP,22
211,"АО ""Дальтрансуголь""",NTC,МК «СУЭК ЛТД» (ООО),SPV,2000000.0,USD,2000000.0,2024-08-30,243,6M-1Y,NaN,NaN,NaN,NaN,NaN,SAP,211


In [17]:
douple_srings_data['E_C'] = douple_srings_data.Entity +douple_srings_data.Counterparty + douple_srings_data.amoutn_outstanding.abs().astype(str) +douple_srings_data.termEnd.astype(str)
douple_srings_data['C_E'] = douple_srings_data.Counterparty +douple_srings_data.Entity +douple_srings_data.amoutn_outstanding.abs().astype(str) +douple_srings_data.termEnd.astype(str)

for value in douple_srings_data.E_C:
    if douple_srings_data.C_E.isin([value]).any():
       index = douple_srings_data[douple_srings_data['E_C'] == value].index[0]
       douple_srings_data = douple_srings_data.drop(index, axis = 0)
    else:
        continue
douple_srings_data.shape
douple_srings_data

,Entity,Entity_group,Counterparty,Counterparty_Group,amoutn_outstanding,Currency,amount_USD_eq,termEnd,Days,Period,dealClass1,dealClass2,instrumentOwner,dealSet,facility,Source,index_copy,E_C,C_E
211,"АО ""Дальтрансуголь""",NTC,МК «СУЭК ЛТД» (ООО),SPV,2.000000e+06,USD,2.000000e+06,2024-08-30,243,6M-1Y,NaN,NaN,NaN,NaN,NaN,SAP,211,"АО ""Дальтрансуголь""МК «СУЭК ЛТД» (ООО)2000000....","МК «СУЭК ЛТД» (ООО)АО ""Дальтрансуголь""2000000...."
14,SUEK AG (Swiss),SUEK INT,SUEK Assets Holding AG,SUEK INT,5.000000e+06,USD,5.000000e+06,2023-12-31,0,<1M,NaN,NaN,NaN,NaN,NaN,SAP,14,SUEK AG (Swiss)SUEK Assets Holding AG5000000.0...,SUEK Assets Holding AGSUEK AG (Swiss)5000000.0...
215,"АО ""Разрез Тугнуйский""",SUEK RU,МК «СУЭК ЛТД» (ООО),SPV,6.545000e+06,EUR,6.888849e+06,2027-12-22,1452,2Y+,NaN,NaN,NaN,NaN,NaN,SAP,215,"АО ""Разрез Тугнуйский""МК «СУЭК ЛТД» (ООО)65450...","МК «СУЭК ЛТД» (ООО)АО ""Разрез Тугнуйский""65450..."
315,МК «СУЭК ЛТД» (ООО),SPV,"ООО ""СУЭК-Хакасия""",SUEK RU,-9.550000e+06,EUR,-1.005172e+07,2027-12-21,1451,2Y+,NaN,NaN,NaN,NaN,NaN,SAP,315,"МК «СУЭК ЛТД» (ООО)ООО ""СУЭК-Хакасия""9550000.0...","ООО ""СУЭК-Хакасия""МК «СУЭК ЛТД» (ООО)9550000.0..."
326,"АО ""СУЭК""",SUEK RU,ООО «АПК «Сибирь»,SUEK RU,-9.828506e+06,RUB,-1.013229e+05,2030-10-20,2485,2Y+,NaN,NaN,NaN,NaN,NaN,SAP,326,"АО ""СУЭК""ООО «АПК «Сибирь»9828505.62030-10-20","ООО «АПК «Сибирь»АО ""СУЭК""9828505.62030-10-20"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,"АО ""СУЭК""",SUEK RU,"ООО ""СГК""",SGC,-2.639882e+10,RUB,-2.721477e+08,2031-12-31,2922,2Y+,NaN,NaN,NaN,NaN,NaN,SAP,279,"АО ""СУЭК""ООО ""СГК""26398815840.852031-12-31","ООО ""СГК""АО ""СУЭК""26398815840.852031-12-31"
276,"АО ""СУЭК""",SUEK RU,"ООО ""СГК""",SGC,-3.500000e+10,RUB,-3.608180e+08,2024-09-26,270,6M-1Y,NaN,NaN,NaN,NaN,NaN,SAP,276,"АО ""СУЭК""ООО ""СГК""35000000000.02024-09-26","ООО ""СГК""АО ""СУЭК""35000000000.02024-09-26"
277,"АО ""СУЭК""",SUEK RU,"ООО ""СГК""",SGC,-3.645621e+10,RUB,-3.758303e+08,2026-03-25,815,2Y+,NaN,NaN,NaN,NaN,NaN,SAP,277,"АО ""СУЭК""ООО ""СГК""36456213309.02026-03-25","ООО ""СГК""АО ""СУЭК""36456213309.02026-03-25"
268,"АО ""Кузбассэнерго""",SGC,"ООО ""СГК""",SGC,5.046926e+10,RUB,5.202920e+08,2027-06-27,1274,2Y+,NaN,NaN,NaN,NaN,NaN,SAP,268,"АО ""Кузбассэнерго""ООО ""СГК""50469256110.772027-...","ООО ""СГК""АО ""Кузбассэнерго""50469256110.772027-..."


In [18]:
douple_srings_data_2['E_C'] = douple_srings_data_2.Entity +douple_srings_data_2.Counterparty + douple_srings_data_2.amoutn_outstanding.abs().astype(str) +douple_srings_data_2.termEnd.astype(str)
douple_srings_data_2['C_E'] = douple_srings_data_2.Counterparty +douple_srings_data_2.Entity +douple_srings_data_2.amoutn_outstanding.abs().astype(str) +douple_srings_data_2.termEnd.astype(str)

for value in douple_srings_data_2.E_C:
    if douple_srings_data_2.C_E.isin([value]).any():
       index = douple_srings_data_2[douple_srings_data_2['E_C'] == value].index[0]
       douple_srings_data_2 = douple_srings_data_2.drop(index, axis = 0)
    else:
        continue
douple_srings_data_2.shape


(38, 19)

In [19]:
names_dictionary = dict(zip(names_dict.keys(), [i[0] for i in names_dict.values()]))
with pd.ExcelWriter(file, engine='openpyxl', mode='a') as writer:
    douple_srings_data.to_excel(writer,sheet_name='Copies_del', index=False)
    douple_srings_data_2.to_excel(writer,sheet_name='Copies_same_del', index=False)
    # data_not_copy.to_excel(writer,sheet_name='Not_copies', index=False)
    # pd.DataFrame({'keys': names_dictionary.keys(), 'values': names_dictionary.values()}).to_excel(writer,sheet_name='Copies_dict', index=False)